 # STAGE C TAG-ALONG PROJECT
## Stability of the Grid System
 
### anthonynwachukwu19@gmail.com


__Instruction on how to manage dataset__.

__Predictive features:__

'tau1' to 'tau4': the reaction time of each network participant, a real value within the range 0.5 to 10 ('tau1' corresponds to the supplier node, 'tau2' to 'tau4' to the consumer nodes); 'p1' to 'p4': nominal power produced (positive) or consumed (negative) by each network participant, a real value within the range -2.0 to -0.5 for consumers ('p2' to 'p4'). As the total power consumed equals the total power generated, p1 (supplier node) = - (p2 + p3 + p4); 'g1' to 'g4': price elasticity coefficient for each network participant, a real value within the range 0.05 to 1.00 ('g1' corresponds to the supplier node, 'g2' to 'g4' to the consumer nodes; 'g' stands for 'gamma');

__Dependent variables:__

_'stab'_: the maximum real part of the characteristic differential equation root (if positive, the system is linearly unstable; if negative, linearly stable); 'stabf': a categorical (binary) label ('stable' or 'unstable'). Because of the direct relationship between 'stab' and 'stabf' ('stabf' = 'stable' if 'stab' <= 0, 'unstable' otherwise), 'stab' should be dropped and 'stabf' will remain as the sole dependent variable (binary classification).

Split the data into an 80-20 train-test split with a random state of “1”. Use the standard scaler to transform the train set (x_train, y_train) and the test set (x_test). Use scikit learn to train a random forest and extra trees classifier. And use xgboost and lightgbm to train an extreme boosting model and a light gradient boosting model. Use random_state = 1 for training all models and evaluate on the test set.

Also, to improve the Extra Trees Classifier, you will use the following parameters (number of estimators, minimum number of samples, minimum number of samples for leaf node and the number of features to consider when looking for the best split) for the hyperparameter grid needed to run a Randomized Cross Validation Search (RandomizedSearchCV).

n_estimators = [50, 100, 300, 500, 1000]

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None]

hyperparameter_grid = {'n_estimators': n_estimators,

'min_samples_leaf': min_samples_leaf,

'min_samples_split': min_samples_split,

'max_features': max_features}__

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#import data set
df = pd.read_csv('C:/Users/TonyC/Desktop/Intern Projects/Managing the Quality Metric of Global Ecological Footprint/Data.csv')

In [3]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [4]:
df.isnull().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [5]:
df.shape

(10000, 14)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


In [7]:
df['stabf'].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [8]:
df.drop("stab",axis=1, inplace=True)

In [9]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [10]:
#splitting variables
x=df.drop('stabf',axis=1)
y=df['stabf']

In [11]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2 , random_state= 1 )

In [12]:
#Scaling dataset using standard scaler
# train datset
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_df = scaler.fit_transform(x_train)
train_df = pd.DataFrame(train_df, columns=x_train.columns)

In [13]:
train_df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,0.367327,-0.986042,0.650447,1.547527,-0.291490,0.061535,1.293862,-0.845074,0.160918,0.339859,0.585568,0.492239
1,-0.064659,0.089437,1.035079,-1.641494,0.619865,-0.067235,-1.502925,0.486613,-0.293143,-1.558488,1.429649,-1.443521
2,-1.467850,1.298418,-0.502536,1.166046,-0.180521,0.490603,0.682560,-0.855302,1.399350,1.451534,-1.045743,0.492489
3,0.820081,0.529920,1.299657,-1.141975,-0.812854,-0.763632,1.521579,0.658780,-0.958319,1.361958,1.604140,0.275303
4,0.665424,-1.425627,0.312300,0.919137,-1.614296,0.760315,1.422019,0.639243,1.676895,0.695660,1.137504,-1.312575


In [14]:
# scaling test data set
test_df = scaler.fit_transform(x_test)
test_df = pd.DataFrame(test_df, columns=x_test.columns)

In [15]:
test_df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,0.592163,-0.420565,1.472472,1.093036,0.426786,-1.504594,-0.792677,1.600201,-0.925703,1.175287,-1.492644,1.086291
1,0.199183,0.364543,-0.190076,-0.518473,-0.229402,-1.071766,0.427103,1.052337,-1.655910,-0.395949,1.412703,1.227535
2,-1.086035,-0.321834,-0.873505,0.011761,-0.977094,0.094896,0.813041,0.751381,1.450284,-1.444370,0.654216,-1.679799
3,-0.087014,-1.113357,0.361518,-1.684316,0.792280,-1.649041,0.410662,-0.084473,0.066085,-1.679450,-0.349573,1.057439
4,0.873004,1.425833,0.080476,1.681022,-0.154247,-0.024315,-0.197525,0.485988,0.119716,-1.475773,0.957057,-0.817608


In [16]:
import imblearn
#overfitting
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_resample(x_train, y_train)
#value count our y_balanced to cross check the effect of smote on the dataset
y_balanced.value_counts()

stable      5092
unstable    5092
Name: stabf, dtype: int64

__RANDOM FOREST__

In [17]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(train_df, y_train)

RandomForestClassifier()

In [18]:
y_pred_rfc = random_forest.predict(test_df)
y_pred_rfc

array(['unstable', 'unstable', 'stable', ..., 'stable', 'stable',
       'unstable'], dtype=object)

In [19]:
#Confusion Matrix for Random Forest
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
cnf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred_rfc, labels=[ 'stable', 'unstable' ])
#Preview the confusion matrix
good_cnf = pd.DataFrame(cnf_mat, columns=['Actual- Positive', 'Actual- Negative'], 
                        index=['Predicted- Positive', 'Predicted- Negative'])
good_cnf

,Actual- Positive,Actual- Negative
Predicted- Positive,618,94
Predicted- Negative,65,1223


In [20]:
#Random Forest Accuracy
accuracy = accuracy_score(y_test,y_pred_rfc)
precision = precision_score(y_test, y_pred_rfc, pos_label='unstable')
recall = recall_score(y_test, y_pred_rfc, pos_label='unstable')
f1 = f1_score(y_test, y_pred_rfc, pos_label='unstable')

print('F1_Score: {}'.format(round(f1*100), 2),"%")
print('Recall: {}'.format(round(recall*100), 2),"%")
print('Precision: {}'.format(round(precision*100),2),"%")
print(f"Accuracy: {round(accuracy*100,2)}","%")

F1_Score: 94 %
Recall: 95 %
Precision: 93 %
Accuracy: 92.05 %


__XGBOOST__.

In [21]:
#xgboost classifier
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state=1)
xgb.fit(train_df, y_train)

C:\Users\TonyC\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:36:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [22]:
y_pred_xgb = xgb.predict(test_df)

In [23]:
#Confusion Matrix for XGB
cnf_mat_xgb = confusion_matrix(y_true=y_test, y_pred=y_pred_xgb, labels=[ 'stable', 'unstable'])
#Preview the confusion matrix
good_cnf = pd.DataFrame(cnf_mat_xgb, columns=['Actual- Positive', 'Actual- Negative'], 
                        index=['Predicted- Positive', 'Predicted- Negative'])
good_cnf


,Actual- Positive,Actual- Negative
Predicted- Positive,647,65
Predicted- Negative,43,1245


In [24]:
# Metrics Calculation for XGBoost 
accuracy = accuracy_score(y_test,y_pred_xgb)
precision = precision_score(y_test, y_pred_xgb, pos_label='unstable')
recall = recall_score(y_test, y_pred_xgb, pos_label='unstable')
f1 = f1_score(y_test, y_pred_xgb, pos_label='unstable')

print(f'F1_Score: {round(f1*100,2)}','%')
print(f'Recall: {round(recall*100,2)}','%')
print(f'Precision: {round(precision*100,2)}','%')
print(f"Accuracy: {round(accuracy*100,2)}")

F1_Score: 95.84 %
Recall: 96.66 %
Precision: 95.04 %
Accuracy: 94.6


In [25]:
# Another method to calculate f1 score
F1 = 2 * (precision * recall) / (precision + recall)
print(f'F1_Score: {round(F1*100,2)}','%')

F1_Score: 95.84 %


`

__Light GBM__

In [26]:
## Lightgbm Model
import lightgbm as lgb
lgb_clf = lgb.LGBMClassifier(random_state=1)
lgb_clf.fit(train_df, y_train)
y_pred_lgb = lgb_clf.predict(test_df)

In [27]:
#Confusion Matrix for XGB
cnf_mat_lgb = confusion_matrix(y_true=y_test, y_pred=y_pred_lgb, labels=[ 'stable', 'unstable' ])
#Preview the confusion matrix
good_cnf = pd.DataFrame(cnf_mat_lgb, columns=['Actual- Positive', 'Actual- Negative'], 
                        index=['Predicted- Positive', 'Predicted- Negative'])
good_cnf

,Actual- Positive,Actual- Negative
Predicted- Positive,636,76
Predicted- Negative,51,1237


In [28]:
# Metrics calculation for LGM
accuracy = accuracy_score(y_test,y_pred_lgb)
precision = precision_score(y_test, y_pred_lgb, pos_label='unstable')
recall = recall_score(y_test, y_pred_lgb, pos_label='unstable')
F1 = f1_score(y_test, y_pred_lgb, pos_label='unstable')

print(f'F1_score: {round(F1*100,2)}','%')
print(f'Recall: {round(recall*100,2)}','%')
print(f'Precision: {round(precision*100,2)}','%')
print(f"Accuracy: {round(accuracy*100,4)}","%")

F1_score: 95.12 %
Recall: 96.04 %
Precision: 94.21 %
Accuracy: 93.65 %


__Extra Trees Classifier__

In [29]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(random_state = 1)
etc.fit(train_df, y_train)
y_pred_etc = etc.predict(test_df)

In [30]:
#Confusion Matrix for ETC
cnf_mat_etc = confusion_matrix(y_true=y_test, y_pred=y_pred_etc, labels=[ 'stable', 'unstable' ])
#Preview the confusion matrix
good_cnf = pd.DataFrame(cnf_mat_etc, columns=['Actual- Positive', 'Actual- Negative'], 
                        index=['Predicted- Positive', 'Predicted- Negative'])
good_cnf

,Actual- Positive,Actual- Negative
Predicted- Positive,602,110
Predicted- Negative,38,1250


In [31]:
# XGB Metric Calculation
Accuracy = accuracy_score(y_test, y_pred_etc)
Precision = precision_score(y_test, y_pred_etc, pos_label='unstable')
Recall = recall_score(y_test, y_pred_etc, pos_label='unstable')
F1 = f1_score(y_test, y_pred_etc, pos_label='unstable')
print(f'F1_Score: {round(F1*100,2)}','%')
print(f'Recall: {round(Recall*100,2)}','%')
print(f'Precision: {round(precision*100,2)}','%')
print(f'Accuracy: {round(Accuracy*100,2)}','%')

F1_Score: 94.41 %
Recall: 97.05 %
Precision: 94.21 %
Accuracy: 92.6 %


__Tuning Parameters__

In [32]:
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]
hyperparameter_grid = {'n_estimators': n_estimators,
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}

__Random CrossValidation Search__

In [33]:
from sklearn.model_selection import RandomizedSearchCV

In [34]:
#Running search on selected number of parameter on train_df and y_train
ran_search = RandomizedSearchCV(estimator = etc,param_distributions=hyperparameter_grid,random_state=1)
ran_search = ran_search.fit(train_df, y_train)

In [35]:
ran_search.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 8,
 'max_features': None}

__New Extra Tree Classifier with recommended Parameters__

In [ ]:
n_classifier = ExtraTreesClassifier(random_state=1,n_estimators = 1000, 
                                    min_samples_split=2,min_samples_leaf= 8,
                                    max_features=None)
                                       
n_classifier.fit(train_df, y_train)
new_pred = n_classifier.predict(test_df)

In [ ]:
# New ETC Accuracy,Recall,precision and F1 score
Accuracy = accuracy_score(y_test, new_pred)
Recall=recall_score(y_test, new_pred,pos_label='unstable')
Precision=precision_score(y_test, new_pred,pos_label='unstable')
F1=f1_score(y_test,new_pred,pos_label='unstable')

print(f'Accuracy: {round(Accuracy*100,2)}','%')
print(f'Precision: {round(Precision*100,2)}','%')
print(f'Recall: {round(Recall*100,2)}','%')
print(f'F1_score: {round(F1*100,2)}','%')

__Feature Importance__

In [ ]:
#Sorting how important features are to models
imp = ran_search.best_estimator_.feature_importances_

sorted(zip(imp,x),reverse=True)

__tau2 having the most importance and p1 having the least importance__

## END OF NOTEBOOK